In [2]:
import numpy as np
import pandas as pd

# GA County Pop

In [4]:
county = pd.read_csv('county_pop.csv')
print(county.columns)

Index(['fips', 'pop2025', 'pop2020', 'state', 'stateCode', 'growthSince2020',
       'name', 'county', 'slug', 'areaMi', 'densityMi', 'rank'],
      dtype='object')


In [5]:
highest_density = county.loc[county['densityMi'].idxmax()]
lowest_density = county.loc[county['densityMi'].idxmin()]
random_density = county.loc[county['densityMi'].sample().index[0]]
print(f"Highest density: {highest_density['county']} with {highest_density['densityMi']} people per square mile")
print(f"Lowest density: {lowest_density['county']} with {lowest_density['densityMi']} people per square mile")
print(f"Random County: {random_density['county']} with {random_density['densityMi']} people per square mile")

Highest density: DeKalb County with 2856.0373134328356 people per square mile
Lowest density: Baker County with 7.733918128654971 people per square mile
Random County: Hart County with 124.55172413793105 people per square mile


In [6]:
file_path = 'populations.csv'
def init_state(file_path):
    input = pd.read_csv(file_path)
    input = pd.DataFrame(np.array([input.values.flatten(),[0]*input.shape[0],[0]*input.shape[0]]).T,columns=['S','I','R'])
    location = np.where(input['S'] == input['S'].max())[0][0]
    input.loc[location, "I"] = 250
    return input

In [7]:
init_state(file_path).to_csv('init_state.csv', index=False)

In [4]:
county = county.drop(columns=['fips','pop2020','state','stateCode','growthSince2020','slug','areaMi','rank','name','densityMi'])
print(county.columns)
print(county.head)

Index(['pop2025', 'county'], dtype='object')
<bound method NDFrame.head of      pop2025             county
0    1089919      Fulton County
1     998232    Gwinnett County
2     786549        Cobb County
3     765418      DeKalb County
4     308915     Chatham County
..       ...                ...
154     2853        Clay County
155     2645       Baker County
156     2337     Webster County
157     2322     Quitman County
158     1611  Taliaferro County

[159 rows x 2 columns]>


In [5]:
# order populations same as transition matrix
# sort populations alphabetically
county = county.sort_values(by='county').drop(columns=['county'])
county.to_csv("populations.csv", index=False)

# US County Pop

In [47]:
counties = pd.read_csv('covid_county_population_usafacts.csv')
print(counties.columns)

Index(['countyFIPS', 'County Name', 'State', 'population'], dtype='object')


In [48]:
# filter for Georgia counties
ga_counties = counties.loc[counties['State'] == 'GA']
# drop unnecessary columns
ga_counties = ga_counties.drop(columns=['State', 'countyFIPS'])
# drop rows with missing population
ga_counties = ga_counties[ga_counties['population'] != 0]
ga_counties.reset_index(drop=True, inplace=True)
ga_pop = ga_counties['population'].values.sum()
print(f"Total population in Georgia counties: {ga_pop}")

Total population in Georgia counties: 10617423


In [49]:
ga_counties.to_csv('populations.csv', index=False)

In [50]:
# infected counties 
infected_counties = {'Bartow County': 3, 'Charlton County': 1, 'Cherokee County': 2, 'Cobb County': 8, 'DeKalb County': 5, 'Fayette County': 4, 'Floyd County': 1, 'Fulton County': 8, 'Gordon County': 2, 'Gwinnett County': 2, 'Lee County': 1, 'Lowndes County': 1, 'Polk County': 1}
# add SIR columns to the dataframe
ga_counties['S'] = ga_counties['population']
ga_counties['I'] = 0
ga_counties['R'] = 0
# set infected counties to the number of infected
for county, infected in infected_counties.items():
    ga_counties.loc[ga_counties['County Name'] == county, 'I'] = infected
    ga_counties.loc[ga_counties['County Name'] == county, 'S'] -= infected


In [42]:
# get index of fulton county
fulton_index = ga_counties[ga_counties['County Name'] == 'Fulton County'].index[0]
print(fulton_index)

59


In [51]:
appling_index = ga_counties[ga_counties['County Name'] == 'Appling County'].index[0]
print(appling_index)

0


In [43]:
ga_counties = ga_counties.drop(columns=['County Name', 'population'])

In [44]:
ga_counties['I'].sum()

39

In [45]:
ga_counties.to_csv('init_state.csv', index=False)